# Simple band combination 

Creates a simple RGB composite 

The CWL workflow has four parameters:

* input_reference
* the common band names for the RGB red, green and blue channels
* aoi
* color

In [9]:
s2_identifier = 'S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720'

In [10]:
input_reference = f'https://terradue-stac.gitlab.io/sentinel-s2-l2a-cogs/53/H/PA/{s2_identifier}/{s2_identifier}.json'

In [11]:
bands = ['red', 'green', 'blue']

In [12]:
aoi = 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))'

In [13]:
color = 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.45'

In [24]:
import os
import sys
import yaml
from shapely.geometry import shape
from ipyleaflet import Map
from pystac import read_file
sys.path.append('../..')
from helpers import stage, item_to_img_overlay, my_read_method

## Stage-in remote STAC items as a local STAC Catalog

In [16]:
catalogs = stage([input_reference])

* <Catalog id=S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720>
  * <Item id=S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720>


In [17]:
catalogs

['/workspace/scombi-do/demo/cwl-workflows/raster/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720']

## Update the parameters file for the CWL execution

In [18]:
with open(r'scombi-do-raster.yml') as file:
    
    cwl_params = yaml.load(file, Loader=yaml.FullLoader)

In [19]:
cwl_params['input_reference']['path'] = catalogs[0]

for index, channel in enumerate(['red-band', 'green-band', 'blue-band']):

    cwl_params[channel] = bands[index]
    
cwl_params['aoi'] = aoi
cwl_params['color'] = color

In [20]:
cwl_params

{'aoi': 'POLYGON((136.707 -35.991,136.707 -35.804,137.071 -35.804,137.071 -35.991,136.707 -35.991))',
 'blue-band': 'blue',
 'color': 'Gamma RGB 3.5 Saturation 1.4 Sigmoidal RGB 15 0.45',
 'green-band': 'green',
 'input_reference': {'class': 'Directory',
  'path': '/workspace/scombi-do/demo/cwl-workflows/raster/S2B_MSIL2A_20200209T004659_N0214_R102_T53HPA_20200209T022720'},
 'red-band': 'red'}

In [21]:
with open(r'scombi-do-raster.yml', 'w') as file:
    params = yaml.dump(cwl_params, file)

## Run the CWL document

Open a Terminal and run the CWL document with:

```console
cd /home/jovyan/demo/cwl-workflows/raster
cwltool --no-container scombi-do-raster.cwl#scombi-do-raster scombi-do-raster.yml 
```

Use the `demo/cwl-result-viewer.ipynb` Jupyter Notebook to view the results

## Inspect the results

Paste the CWL execution path returned by the successful run

Example:

```console
...
  }
        ],
        "path": "/home/jovyan/test-workflows/expert-mode/38p2k39r"
    }
}
INFO Final process status is success
```

In [22]:
result_dir = '/workspace/scombi-do/demo/cwl-workflows/raster/7z1g_kts'

In [ ]:
item = next(read_file(os.path.join(result_dir, 'catalog.json')).get_items())

m = Map(center=(shape(item.geometry).centroid.y, 
                shape(item.geometry).centroid.x), 
                zoom=11)

image = item_to_img_overlay(item)

m.add_layer(image)
m